In [56]:
import numpy as np
import pandas as pd

In [63]:
def prepare_df(runname):
    df = pd.read_csv(f'data/{runname}/df_all_clean_vehicles.csv', header=list(range(2)), index_col=[0, 1])
    df.columns = [': '.join(col) for col in df.columns]
    df.index.set_names(['Scenario ID', 'Vehicle'], inplace=True)
    df = df.reset_index()
    
    df = df[df['Violation type: Priority violation']]
    df = df[df['Vehicle'] == 'V0']
    
    df = pd.concat([
        df['Scenario ID'].str.extract('(?P<scenario_minus_forcing>.+), forcing (?P<forcing>[^,]+)'),
        df[[
            'Vehicle',
            'Output of simulation (execution): No. of collisions',
        ]],
    ], axis=1)
    
    df = df.pivot(
        index=['scenario_minus_forcing', 'Vehicle'],
        columns=['forcing'],
        values=['Output of simulation (execution): No. of collisions'],
    )
    df.columns = [col[1] for col in df.columns]
    
    return df


def compute_percentages(df):    
    # Initialize counts
    best_count = 0
    equal_to_best_count = 0
    worst_count = 0
    equal_to_worst_count = 0
    
    # Iterate through each row
    total_rows = df.shape[0]
    for _, row in df.iterrows():
        values = row.values
        last_col_value = row['stops as recommended']
        max_value = np.max(values)
        min_value = np.min(values)
        
        if last_col_value == min_value:
            if np.sum(values == min_value) == 1:
                best_count += 1
            else:
                equal_to_best_count += 1    
        elif last_col_value == max_value:
            if np.sum(values == max_value) == 1:
                worst_count += 1
            else:
                equal_to_worst_count += 1            
    
    # Compute percentages
    best_percentage = (best_count / total_rows) * 100
    equal_to_best_percentage = (equal_to_best_count / total_rows) * 100
    worst_percentage = (worst_count / total_rows) * 100
    equal_to_worst_percentage = (equal_to_worst_count / total_rows) * 100
    moderate_percentage = 100 - (best_percentage + equal_to_best_percentage + worst_percentage + equal_to_worst_percentage)
    
    # Print Results
    print(f"Better than Others: {best_percentage:.1f}%")
    print(f"Equal to the Best: {equal_to_best_percentage:.1f}%")
    print(f"Moderate: {moderate_percentage:.1f}%")
    print(f"Equal to the Worst: {equal_to_worst_percentage:.1f}%")
    print(f"Worse than Others: {worst_percentage:.1f}%")


def make_df_cmp():
    df = pd.concat([
        prepare_df('20250228_090518'), 
        prepare_df('20250301_173919_stops_as_recommended'), 
    ], axis=1)    
    
    cols = list(df)
    cols.pop(cols.index('ignoring human'))
    cols.pop(cols.index('stops 50% with forcing 50%'))
    df = df[cols]
    
    compute_percentages(df)
    
    return df


df_cmp = make_df_cmp()
df_cmp

Better than Others: 30.0%
Equal to the Best: 16.5%
Moderate: 40.5%
Equal to the Worst: 5.0%
Worse than Others: 8.0%


,,change of priorities,stops,stops 50%,stops as recommended
scenario_minus_forcing,Vehicle,,,,
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no",V0,7.0,5.0,4.0,4.0
"map-generator/generated-maps/3_with_bridges/scenario1-10.json, passhum 0, slowness no",V0,2.0,6.0,7.0,1.0
"map-generator/generated-maps/3_with_bridges/scenario1-2.json, passhum 0, slowness no",V0,7.0,10.0,5.0,5.0
"map-generator/generated-maps/3_with_bridges/scenario1-3.json, passhum 0, slowness no",V0,2.0,2.0,4.0,2.0
"map-generator/generated-maps/3_with_bridges/scenario1-4.json, passhum 0, slowness no",V0,1.0,5.0,4.0,2.0
...,...,...,...,...,...
"map-generator/generated-maps/3_without_bridges/scenario9-5.json, passhum 0, slowness no",V0,10.0,14.0,10.0,11.0
"map-generator/generated-maps/3_without_bridges/scenario9-6.json, passhum 0, slowness no",V0,10.0,13.0,13.0,12.0
"map-generator/generated-maps/3_without_bridges/scenario9-7.json, passhum 0, slowness no",V0,12.0,15.0,14.0,13.0
